In [1]:
%display latex

In [2]:
N = 3
F = GF(2^3, 'x')
x = F.gen()

In [3]:
basis = [x^3, x^5, x^6] # self-dual basis

In [4]:
# express an element as a linear combination of the basis
components = lambda k: [(k * el).trace() for el in basis]

# additive group character
chi = lambda k: exp(pi * I * int(k.trace()))

def toInt(k):
    return list(F).index(k)

In [5]:
y = var('y')

# compute the coefficients for the basis elements,
def solve_basis(basis, curve):
    sols = []
    for k in basis:
        sol = solve(chi(k * curve(k)) == y^2, y, solution_dict=True)
        sols.append(sol[1]) # positive solutions only (choice)
    return sols

In [6]:
# after computing the solutions we can simply
# get them for each basis element.
def basis_c(a, l, sols):
    if a == 0:
        return 1
    return sols[l][y]

In [7]:
# Compute an arbitrary coefficient c_{\alpha,f} for a 
# given curve f using the general formula.
def c(alpha, curve, sols=None):
    if not sols:
        sols = solve_basis(basis, curve)
    
    # Expand alpha in the basis
    comps = components(alpha)

    # Apply the formula
    s1 = 0
    for k in range(N-1):
        s2 = 0
        for j in range(k+1, N):
            s2 += comps[j] * basis[j]
        s1 += s2 * curve(comps[k] * basis[k])
        
    return chi(s1) * prod([basis_c(a, l, sols) for l, a in enumerate(comps)])

In [8]:
def sign_perm(sols, perm=None):
    if perm:
        for k, sol in enumerate(sols):
            sols[k][y] = perm[k] * sols[k][y]
    return sols

In [9]:
for mu in F:
    curve = lambda t: mu * t # \beta = \mu \alpha
    sols = sign_perm(solve_basis(basis, curve), [1,1,1])
    for k in F:
        for kp in F:
            lhs = c(k, curve, sols) * c(kp, curve, sols)
            rhs = chi(kp * curve(k)) * c(k + kp, curve, sols)
            if lhs != rhs:
                raise Exception('Recurrence relation does not hold!', mu)
print('Recurrence relation holds!')

Recurrence relation holds!


In [10]:
def ray(mu):
    return lambda t: mu * t

In [11]:
def PS(curve, perms):
    phase_space = zero_matrix(SR, 2^N, 2^N)
    phase_space[0,:] = 1 # vertical line
    for j, mu in enumerate(F): # iterate through the curve parametr
        # same sign choice for a fixed curve parameter
        sols = sign_perm(solve_basis(basis, curve(mu)), perms[j])
        for i, a in enumerate(F):
            # loop through alpha to obtain coefficient and
            # corresponding point
            coeff = c(a, curve(mu), sols)
            b = curve(mu)(a)
            phase_space[i, toInt(b)] = coeff
    return phase_space

In [12]:
ps = PS(ray, [[1,1,1]] * 8)
ps

[ 1  1  1  1  1  1  1  1]
[ 1 -1 -I  1 -I  I  I -1]
[ 1 -I -1  I -I  I  1 -1]
[ 1  1  I  I  I  1  1  I]
[ 1 -I -I  I -1  1  I -1]
[ 1  I  I  1  1  I  1  I]
[ 1  I  1  1  I  1  I  I]
[ 1 -1 -1  I -1  I  I -I]

In [13]:
def Proj(u, v=None):
    if not v:
        v = u
    return u.tensor_product(v.conjugate_transpose())

Id = identity_matrix(2^N)

def Fourier():
    s = zero_matrix(SR, 2^N, 2^N)
    for i, a in enumerate(F):
        for j, b in enumerate(F):
            s[i,j] = chi(a * b) / sqrt(2^N)
    return s
FF = Fourier()

In [14]:
def phi(a, b):
    return ps[toInt(a), toInt(b)]

def Z(a):
    return diagonal_matrix([chi(a * k) for k in F])

def X(b):
    return FF.conjugate_transpose() * Z(b) * FF

def D(a, b):
    return phi(a, b) * Z(a) * X(b)

The displacement operators should be trace orthogonal, null trace, unitary and hermititan.

In [15]:
# trace orthogonal
for a in F:
    for b in F:
        D1 = D(a, b)
        for c in F:
            for d in F:
                D2 = D(c, d)
                tr = (D1 * D2.conjugate_transpose()).trace()
                dl = int(a == c) * int(b == d)
                if tr != (2^N) * dl:
                    raise Exception

In [16]:
# null trace
for a in F:
    for b in F:
        if a != 0 and b != 0:
            d = D(a, b)
            if d.trace() != 0:
                raise Exception(a, b)

In [17]:
# hermitian
for a in F:
    for b in F:
        d = D(a, b)
        if d != d.conjugate_transpose():
            raise Exception

In [18]:
# unitarity
for a in F:
    for b in F:
        d = D(a,b)
        if d * d.conjugate_transpose() != Id:
            raise Exception

In [19]:
def graph(points):
    m = zero_matrix(SR, 2^N)
    for p in points:
        m[toInt(p[0]), toInt(p[1])] = 1
    return m

The property that we want to test is the following:
$$
D(\alpha_1,f(\alpha_1)) D(\alpha_2,f(\alpha_2))
= D(\alpha_1+\alpha_2, f(\alpha_1+\alpha_2))
= D(\alpha_1+\alpha_2, f(\alpha_1)+f(\alpha_2)),
$$
for a given curve $f$ and all $\alpha \in F$. Or for a set of points which gives a curve $\{\alpha,\beta\}$:
$$
D(\alpha_1,\beta_1) D(\alpha_2,\beta_2)
= D(\alpha_1+\alpha_2, \beta_1 + \beta_2).
$$

In [20]:
def testCurve(points):
    for p1 in points:
        for p2 in points:
            p_op = D(p1[0], p1[1]) * D(p2[0], p2[1])
            s_op = D(p1[0] + p2[0], p1[1] + p2[1]) # assuming additivity
            if p_op != s_op:
                raise Exception(p1, p2)
    return True

In [21]:
for mu in F:
    if not testCurve([(k, mu * k) for k in F]):
        raise Exception(mu)

The property holds for all the rays, as it should of course.

Now let's identify all of the family of curves that are valid for three qubits. But first let's test this example of an exceptional curve (equation 5.19 from Annals):
$$
(0,0), (\sigma^4,0), (\sigma^4, \sigma^5), (\sigma^3, \sigma^7), (\sigma^3, \sigma^4), (\sigma^6, \sigma^4), (\sigma^6, \sigma^7), (0, \sigma^5)
$$
This curve can be obtained by the equation:
$$
\beta^2 + \sigma^5 \beta = \sigma^6 \alpha + \sigma^2 \alpha^2,
$$
and the structural equations:
$$
\sigma^6 \alpha + \sigma^4 \alpha^2 + \alpha^4 = 0,
\quad
\sigma^2 \beta + \sigma^6 \beta^2 + \beta^4 = 0.
$$

In [311]:
points = [(F(0),F(0)), (x^4,F(0)), (x^4,x^5), (x^3,x^7), (x^3,x^4), (x^6,x^4), (x^6,x^7), (F(0),x^5)]
points

[(0, 0),
 (x^2 + x, 0),
 (x^2 + x, x^2 + x + 1),
 (x + 1, 1),
 (x + 1, x^2 + x),
 (x^2 + 1, x^2 + x),
 (x^2 + 1, 1),
 (0, x^2 + x + 1)]

In [330]:
points_from_eq = []
for a in F:
    for b in F:
        if b^2 + x^5 * b == x^6 * a + x^2 * a^2:
            if x^6 * a + x^4 * a^2 + a^4 == F(0):
                if x^2 * b + x^6 * b^2 + b^4 == F(0):
                    points_from_eq.append((a,b))

In [331]:
points_from_eq

[(0, 0),
 (0, x^2 + x + 1),
 (x + 1, x^2 + x),
 (x + 1, 1),
 (x^2 + x, 0),
 (x^2 + x, x^2 + x + 1),
 (x^2 + 1, x^2 + x),
 (x^2 + 1, 1)]

In [332]:
set(points) == set(points_from_eq)

True

Just to be sure this curve is additive:

In [325]:
def checkAdditivity(points):
    for p1 in points:
        for p2 in points:
            p3 = (p1[0] + p2[0], p1[1] + p2[1])
            if p3 not in points:
                raise Exception
    return True

def checkIsotropy(points):
    for p1 in points:
        for p2 in points:
            if (p1[0]*p2[1] - p1[1]*p2[0]).trace() != 0:
                raise Exception
    return True

In [313]:
checkAdditivity(points)

True

In [326]:
checkIsotropy(points)

True

In [327]:
graph(points)

[1 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 1]
[1 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 1]
[0 0 0 0 0 0 0 0]

And now we test to see if the set of displacement operators are closed under our rule:

In [315]:
testCurve(points)

True

For example the points $(\sigma^6,\sigma^4)$ and $(\sigma^3,\sigma^7)$ give us the following operators:

In [316]:
phi(x^6,x^4)

I

In [317]:
D(x^6,x^4)

[ 0  0  0  0  I  0  0  0]
[ 0  0 -I  0  0  0  0  0]
[ 0  I  0  0  0  0  0  0]
[ 0  0  0  0  0  0  I  0]
[-I  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  I]
[ 0  0  0 -I  0  0  0  0]
[ 0  0  0  0  0 -I  0  0]

In [318]:
phi(x^3,x^7)

I

In [319]:
D(x^3,x^7)

[ 0  0  0  0  0  0  0  I]
[ 0  0  0  I  0  0  0  0]
[ 0  0  0  0  0  0 -I  0]
[ 0 -I  0  0  0  0  0  0]
[ 0  0  0  0  0 -I  0  0]
[ 0  0  0  0  I  0  0  0]
[ 0  0  I  0  0  0  0  0]
[-I  0  0  0  0  0  0  0]

Their product is:

In [320]:
D(x^6,x^4) * D(x^3,x^7)

[ 0  0  0  0  0  1  0  0]
[ 0  0  0  0  0  0 -1  0]
[ 0  0  0 -1  0  0  0  0]
[ 0  0 -1  0  0  0  0  0]
[ 0  0  0  0  0  0  0  1]
[ 1  0  0  0  0  0  0  0]
[ 0 -1  0  0  0  0  0  0]
[ 0  0  0  0  1  0  0  0]

And the operator given by the sum of their components is:

In [321]:
D(x^6+x^3,x^4+x^7)

[ 0  0  0  0  0  1  0  0]
[ 0  0  0  0  0  0 -1  0]
[ 0  0  0 -1  0  0  0  0]
[ 0  0 -1  0  0  0  0  0]
[ 0  0  0  0  0  0  0  1]
[ 1  0  0  0  0  0  0  0]
[ 0 -1  0  0  0  0  0  0]
[ 0  0  0  0  1  0  0  0]

And as we can see they are equal:

In [322]:
D(x^6,x^4) * D(x^3,x^7) == D(x^6+x^3,x^4+x^7)

True

---
Let me try to calculate the eigenstates. According to the document Klimov sent me we should be able to obtain the eigenstates by summing over the displacement operators indexed by the points of the curve. Seems strange because we are using the phases of rays but...:
$$
\ket{\psi_\kappa^{\alpha,\beta}} \bra{\psi_\kappa^{\alpha,\beta}}
= \frac{1}{2^N} \sum_{\tau} \chi(\kappa\tau) D(\alpha(\tau), \beta(\tau)).
$$

So in order to calculate the eigenvectors I need to calculate the parametric form of the points, which I don't know how to do. But, what if I assume the the points given above are each indexed by a certain value of $\tau$ and just sum over all of them?

In [338]:
def StateFromCurve(curve):
    op = zero_matrix(SR, 2^N, 2^N)
    for j, tau in enumerate(F):
        op += D(*curve[j])
    return op / 2^N

In [335]:
def checkState(state):
    if state.trace() != 1:
        raise Exception('Not unit trace.')
    if state != state.conjugate_transpose():
        raise Exception('Not self-adjoint.')
    return True

We continue with Klimov's calculations, now for the Wigner kernel for the straight lines:
$$
w(\alpha,\beta)
= \frac{1}{2^N} \sum_{\gamma,\delta} \chi(\gamma\beta + \delta\alpha) \phi(\gamma,\delta) Z_\gamma X_\delta.
$$

In [339]:
def Wigner(alpha, beta):
    wigner_kernel = zero_matrix(SR, 2^N, 2^N)
    for j, ga in enumerate(F):
        for k, de in enumerate(F):
            char = chi(ga * beta + de * alpha)
            wigner_kernel += char * D(ga, de) / 2^N
    return wigner_kernel

In [162]:
def WignerMatrix(state):
    m = zero_matrix(SR, 2^N, 2^N)
    for i, a in enumerate(F):
        for j, b in enumerate(F):
            m[i, j] = (Wigner(a, b) * state).trace()
    return m / 2^N

In [336]:
eig_state = StateFromCurve(points)
checkState(state)

True

In [337]:
eig_state

[   1/4      0      0      0  1/4*I    1/4      0  1/4*I]
[     0      0      0      0      0      0      0      0]
[     0      0      0      0      0      0      0      0]
[     0      0      0      0      0      0      0      0]
[-1/4*I      0      0      0    1/4 -1/4*I      0    1/4]
[   1/4      0      0      0  1/4*I    1/4      0  1/4*I]
[     0      0      0      0      0      0      0      0]
[-1/4*I      0      0      0    1/4 -1/4*I      0    1/4]

In [125]:
w = WignerMatrix(eig_state)
w

[1/8   0   0   0   0 1/8   0   0]
[  0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0]
[  0   0   0   0 1/8   0   0 1/8]
[1/8   0   0   0   0 1/8   0   0]
[  0   0   0   0   0   0   0   0]
[  0   0   0   0 1/8   0   0 1/8]
[  0   0   0   0   0   0   0   0]

In [340]:
# is eig_state and eigen state for all ops?
for p in points:
    if D(*p) * eig_state * D(*p).conjugate_transpose() != eig_state:
        raise Exception

In [135]:
# # For external use
# import numpy as np
# np.save('phase_space.npy', ps.n().numpy())

So we have seen that the particular curve given by (5.19) is an abelian curve in order established sense. It is a particular case of 21 exceptional curves that are doubly degenerate, case (i) in the Annals paper. We now wish to see how many of these curves are also abelian.

In [341]:
def ExcCurvesCaseI():
    def ExceptionalCurve(a1, a2):
        b1 = 1/(a1+a2) + 1/a1 + 1/a2
        return [
            (F(0), F(0)),
            (a1, F(0)),
            (a1, b1),
            (a2, b1 * (a1^-1 * a2 + F(1))),
            (a2, b1 * a1^-1 * a2),
            (a1 + a2, b1 * a1^-1 * a2),
            (a1 + a2, b1 * (a1^-1 * a2 + 1)),
            (F(0), b1)
        ]
        
    curves = []
    for a1 in list(F)[1:]:
        for a2 in list(F)[1:]:
            if a1 != a2:
                curves.append(ExceptionalCurve(a1, a2))
    
    curves_uniq = []
    for curve in curves:
        curve.sort()
        if curve not in curves_uniq:
            curves_uniq.append(curve)

    return curves_uniq

In [342]:
curves = ExcCurvesCaseI()
len(curves)

21

In [348]:
for j, curve in enumerate(curves):
    checkAdditivity(curve)
    checkIsotropy(curve)

In [343]:
testCurve(curves[10]) # the 10th curve corresponds to the example 5.19

True

In [349]:
for j, curve in enumerate(curves):
    print('Testing curve #{}...'.format(j))
    try:
        testCurve(curve)
        print('ABELIAN')
    except:
        continue

Testing curve #0...
Testing curve #1...
Testing curve #2...
ABELIAN
Testing curve #3...
Testing curve #4...
ABELIAN
Testing curve #5...
Testing curve #6...
ABELIAN
Testing curve #7...
Testing curve #8...
Testing curve #9...
Testing curve #10...
ABELIAN
Testing curve #11...
Testing curve #12...
Testing curve #13...
Testing curve #14...
ABELIAN
Testing curve #15...
Testing curve #16...
ABELIAN
Testing curve #17...
Testing curve #18...
Testing curve #19...
Testing curve #20...


In [350]:
curves[2]

[(0, 0),
 (0, x + 1),
 (x, 0),
 (x, x + 1),
 (x^2 + 1, 1),
 (x^2 + 1, x),
 (x^2 + x + 1, 1),
 (x^2 + x + 1, x)]

In [351]:
graph(curves[2])

[1 0 0 1 0 0 0 0]
[1 0 0 1 0 0 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 1]
[0 1 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0]

In [352]:
state = StateFromCurve(curves[4])
state

[   1/4      0  1/4*I      0      0      0    1/4  1/4*I]
[     0      0      0      0      0      0      0      0]
[-1/4*I      0    1/4      0      0      0 -1/4*I    1/4]
[     0      0      0      0      0      0      0      0]
[     0      0      0      0      0      0      0      0]
[     0      0      0      0      0      0      0      0]
[   1/4      0  1/4*I      0      0      0    1/4  1/4*I]
[-1/4*I      0    1/4      0      0      0 -1/4*I    1/4]

In [353]:
checkState(state)

True

In [169]:
WignerMatrix(state)

[1/8   0   0   0   0   0 1/8   0]
[  0   0   0   0   0   0   0   0]
[1/8   0   0   0   0   0 1/8   0]
[  0   0 1/8   0   0   0   0 1/8]
[  0   0   0   0   0   0   0   0]
[  0   0 1/8   0   0   0   0 1/8]
[  0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0]

Now for the case (II) exceptional curves. There are seven of these. The following is given by example 5.25 from the annals.

In [354]:
points2 = [(F(0),F(0)), (x^3,F(0)), (x^5,F(0)), (x^2,F(0)), (x^3,x^6), (x^5,x^6), (x^2,x^6), (F(0),x^6)]
points2

[(0, 0),
 (x + 1, 0),
 (x^2 + x + 1, 0),
 (x^2, 0),
 (x + 1, x^2 + 1),
 (x^2 + x + 1, x^2 + 1),
 (x^2, x^2 + 1),
 (0, x^2 + 1)]

In [474]:
powers(points2)

[(0, 0), (3, 0), (5, 0), (2, 0), (3, 6), (5, 6), (2, 6), (0, 6)]

In [475]:
checkAdditivity(points2)
checkIsotropy(points2)

True

In [476]:
testCurve(points2)

True

In [477]:
graph(points2)

[1 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 1 0]
[1 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]

In [173]:
WignerMatrix(StateFromCurve(points2))

[1/8   0   0   0   0   0 1/8   0]
[  0   0   0   0   0   0   0   0]
[1/8   0   0   0   0   0 1/8   0]
[1/8   0   0   0   0   0 1/8   0]
[  0   0   0   0   0   0   0   0]
[1/8   0   0   0   0   0 1/8   0]
[  0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0]

Now we test all seven of these curves.

In [359]:
def ExcCurvesCaseII():
    def ExceptionalCurveII(a1, a2):
        delta = None
        for k in F:
            if (k * a1).trace() == 0 and (k * a2).trace() == 0 and k != F(0):
                delta = k
                break
        return [
            (F(0), F(0)),
            (a1, F(0)),
            (a2, F(0)),
            (a1 + a2, F(0)),
            (a1, delta),
            (a2, delta),
            (a1 + a2, delta),
            (F(0), delta)
        ]

    curves = []
    for a1 in list(F)[1:]:
        for a2 in list(F)[1:]:
            if a1 != a2:
                curves.append(ExceptionalCurveII(a1, a2))
    
    curves_uniq = []
    for curve in curves:
        curve.sort()
        if curve not in curves_uniq:
            curves_uniq.append(curve)

    return curves_uniq

In [360]:
curvesII = ExcCurvesCaseII()
for j, curve in enumerate(curvesII):
    checkAdditivity(curve)
    checkIsotropy(curve)
    try:
        testCurve(curve)
        print('Curve #{} is abelian.'.format(j))
    except:
        continue

Curve #2 is abelian.
Curve #3 is abelian.
Curve #5 is abelian.


In [511]:
curvesII[2]

[(0, 0),
 (0, x + 1),
 (x, 0),
 (x, x + 1),
 (x^2 + 1, 0),
 (x^2 + 1, x + 1),
 (x^2 + x + 1, 0),
 (x^2 + x + 1, x + 1)]

In [512]:
powers(curvesII[3]) # example 5.25

[(0, 0), (0, 6), (3, 0), (3, 6), (2, 0), (2, 6), (5, 0), (5, 6)]

In [513]:
testCurve(curvesII[3])

True

In [363]:
curvesII[5]

[(0, 0),
 (0, x^2 + x + 1),
 (x + 1, 0),
 (x + 1, x^2 + x + 1),
 (x^2 + 1, 0),
 (x^2 + 1, x^2 + x + 1),
 (x^2 + x, 0),
 (x^2 + x, x^2 + x + 1)]

In [183]:
WignerMatrix(StateFromCurve(curvesII[2]))

[1/8   0   0 1/8   0   0   0   0]
[1/8   0   0 1/8   0   0   0   0]
[  0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0]
[1/8   0   0 1/8   0   0   0   0]
[1/8   0   0 1/8   0   0   0   0]
[  0   0   0   0   0   0   0   0]

In [184]:
WignerMatrix(StateFromCurve(curvesII[5]))

[1/8   0   0   0   0 1/8   0   0]
[  0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0]
[1/8   0   0   0   0 1/8   0   0]
[1/8   0   0   0   0 1/8   0   0]
[  0   0   0   0   0   0   0   0]
[1/8   0   0   0   0 1/8   0   0]
[  0   0   0   0   0   0   0   0]

---
Getting the curves from the $(0,9,0)$ factorization using the equation and the structural equations.

In [365]:
pts = []
for a in F:
    for b in F:
        if b^2 + x^5 * b == x^2 * a^2 + x^6 * a:
            if (x^4 * b).trace() == 0:
                if (x^5 * a).trace() == 0:
                    pts.append((a, b))
pts

[(0, 0),
 (0, x^2 + x + 1),
 (x + 1, x^2 + x),
 (x + 1, 1),
 (x^2 + x, 0),
 (x^2 + x, x^2 + x + 1),
 (x^2 + 1, x^2 + x),
 (x^2 + 1, 1)]

In [366]:
for pt in pts:
    if pt not in points:
        raise Exception

In [367]:
pts = []
for a in F:
    for b in F:
        if b^2 + x^2 * b == x^6 * a^2 + x^5 * a:
            if (x^6 * b).trace() == 0:
                if (x^2 * a).trace() == 0:
                    pts.append((a,b))
pts

[(0, 0),
 (0, x^2),
 (x^2, x + 1),
 (x^2, x^2 + x + 1),
 (x^2 + 1, 0),
 (x^2 + 1, x^2),
 (1, x + 1),
 (1, x^2 + x + 1)]

In [204]:
for j, curve in enumerate(curves):
    print(j, '\t', curve)

0 	 [(0, 0), (0, 1), (x, 0), (x, 1), (x^2, x), (x^2, x + 1), (x^2 + x, x), (x^2 + x, x + 1)]
1 	 [(0, 0), (0, x), (1, 1), (1, x + 1), (x, 0), (x, x), (x + 1, 1), (x + 1, x + 1)]
2 	 [(0, 0), (0, x + 1), (x, 0), (x, x + 1), (x^2 + 1, 1), (x^2 + 1, x), (x^2 + x + 1, 1), (x^2 + x + 1, x)]
3 	 [(0, 0), (0, 1), (x, x^2), (x, x^2 + 1), (x^2, 0), (x^2, 1), (x^2 + x, x^2), (x^2 + x, x^2 + 1)]
4 	 [(0, 0), (0, x^2 + 1), (x + 1, 1), (x + 1, x^2), (x^2, 0), (x^2, x^2 + 1), (x^2 + x + 1, 1), (x^2 + x + 1, x^2)]
5 	 [(0, 0), (0, x^2), (1, 1), (1, x^2 + 1), (x^2, 0), (x^2, x^2), (x^2 + 1, 1), (x^2 + 1, x^2 + 1)]
6 	 [(0, 0), (0, x), (1, x^2 + 1), (1, x^2 + x + 1), (x, x^2 + 1), (x, x^2 + x + 1), (x + 1, 0), (x + 1, x)]
7 	 [(0, 0), (0, x^2 + 1), (x + 1, 0), (x + 1, x^2 + 1), (x^2, x), (x^2, x^2 + x + 1), (x^2 + x + 1, x), (x^2 + x + 1, x^2 + x + 1)]
8 	 [(0, 0), (0, x^2 + x + 1), (x + 1, 0), (x + 1, x^2 + x + 1), (x^2 + 1, x), (x^2 + 1, x^2 + 1), (x^2 + x, x), (x^2 + x, x^2 + 1)]
9 	 [(0, 0), (0, 1)

In [368]:
curves[16]

[(0, 0),
 (0, x^2),
 (1, x + 1),
 (1, x^2 + x + 1),
 (x^2, x + 1),
 (x^2, x^2 + x + 1),
 (x^2 + 1, 0),
 (x^2 + 1, x^2)]

In [369]:
testCurve(curves[16])

True

In [370]:
state = StateFromCurve(curves[16])
state

[   1/4      0    1/4  1/4*I      0  1/4*I      0      0]
[     0      0      0      0      0      0      0      0]
[   1/4      0    1/4  1/4*I      0  1/4*I      0      0]
[-1/4*I      0 -1/4*I    1/4      0    1/4      0      0]
[     0      0      0      0      0      0      0      0]
[-1/4*I      0 -1/4*I    1/4      0    1/4      0      0]
[     0      0      0      0      0      0      0      0]
[     0      0      0      0      0      0      0      0]

In [371]:
k = x
state_k = X(k) * state * X(k).conjugate_transpose()

In [219]:
WignerMatrix(state_k)

[  0 1/8   0   0 1/8   0   0   0]
[  0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0 1/8 1/8]
[  0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0]
[  0 1/8   0   0 1/8   0   0   0]
[  0   0   0   0   0   0 1/8 1/8]

In [372]:
# checking if we have valid eigenvectors
state = StateFromCurve(curves[16])
for pt in curves[16]:
    dop = D(*pt)
    for k in F:
        trans_state = X(k) * state * X(k).conjugate_transpose()
        if dop * trans_state * dop.conjugate_transpose() != trans_state:
            raise Exception

---
BUNDLES

In [373]:
def testCurveSet(curves):
    for j, curve in enumerate(curves):
        print('Testing curve #{}...'.format(j))
        try:
            testCurve(curve)
            print('ABELIAN.')
        except:
            continue

First we test the $(3,0,6)$ type bundles.

In [423]:
# other 306 bundles that can be obtained from the rays
# by location transformations
curves306 = [[(F(0),k) for k in F]]
fi = x # 0, x, x^2 and x^4
for phi0 in F:
    curves306.append([
        (k, phi0 * k + fi^2 * k^2 + fi * k^4) for k in F
    ])

In [488]:
testCurve(curves306[2])

True

In [492]:
for p in curves306[2]:
    print(ps[toInt(p[0]), toInt(p[1])])

1
I
-I
I
1
1
I
-1


In [424]:
for curve in curves306:
    checkAdditivity(curve)
    checkIsotropy(curve)

In [425]:
testCurveSet(curves306)

Testing curve #0...
ABELIAN.
Testing curve #1...
Testing curve #2...
ABELIAN.
Testing curve #3...
ABELIAN.
Testing curve #4...
ABELIAN.
Testing curve #5...
Testing curve #6...
Testing curve #7...
ABELIAN.
Testing curve #8...


In [431]:
def powers(pts):
    return [(toInt(p[0]), toInt(p[1])) for p in pts]

In [440]:
powers(curves306[7])

[(0, 0), (1, 0), (2, 3), (3, 3), (4, 3), (5, 0), (6, 0), (7, 3)]

In [441]:
curves306[7]

[(0, 0),
 (x, 0),
 (x^2, x + 1),
 (x + 1, x + 1),
 (x^2 + x, x + 1),
 (x^2 + x + 1, 0),
 (x^2 + 1, 0),
 (1, x + 1)]

In [439]:
graph(curves306[7])

[1 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0]
[0 0 0 1 0 0 0 0]
[0 0 0 1 0 0 0 0]
[1 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0]

In [398]:
state = StateFromCurve(curves306[3])
checkState(state)

True

In [378]:
WignerMatrix(state)

[1/8   0   0   0   0   0   0   0]
[  0   0   0   0   0   0 1/8   0]
[  0 1/8   0   0   0   0   0   0]
[  0   0   0   0   0   0   0 1/8]
[  0   0   0   0   0 1/8   0   0]
[  0   0   0 1/8   0   0   0   0]
[  0   0   0   0 1/8   0   0   0]
[  0   0 1/8   0   0   0   0   0]

So for some $(3,0,6)$ type sets we do actually obtain some abelian curves.

In [397]:
# affine plane
graph_mat = zero_matrix(SR, 2^N, 2^N)
for j, curve in enumerate(curves306):
    for pt in curve:
        graph_mat[toInt(pt[0]), toInt(pt[1])] = j
graph_mat

[8 0 0 0 0 0 0 0]
[7 3 6 8 5 4 2 1]
[6 2 5 7 4 3 8 1]
[3 4 8 7 5 1 6 2]
[1 5 6 7 8 2 3 4]
[7 5 4 2 1 3 6 8]
[7 8 5 4 2 1 3 6]
[5 3 2 7 1 8 4 6]

Now we test the $(1,6,2)$ bundle.

In [458]:
# other 162 bundles that can be obtained from the rays
# by location transformations
curves162 = [[(F(0),k) for k in F]]
fi = x^3 # 1, x+1, x^2+x+1, x^2+1
for phi0 in F:
    curves162.append([
        (k, phi0 * k + fi^2 * k^2 + fi * k^4) for k in F
    ])

In [459]:
for curve in curves162:
    checkAdditivity(curve)
    checkIsotropy(curve)

In [460]:
testCurveSet(curves162)

Testing curve #0...
ABELIAN.
Testing curve #1...
ABELIAN.
Testing curve #2...
Testing curve #3...
Testing curve #4...
Testing curve #5...
Testing curve #6...
Testing curve #7...
Testing curve #8...
ABELIAN.


In [468]:
powers([(k, F(1) * k + fi^2 * k^2 + fi * k^4) for k in F])

[(0, 0), (1, 7), (2, 7), (3, 4), (4, 0), (5, 5), (6, 4), (7, 5)]

In [469]:
[(k, F(1) * k + fi^2 * k^2 + fi * k^4) for k in F]

[(0, 0),
 (x, 1),
 (x^2, 1),
 (x + 1, x^2 + x),
 (x^2 + x, 0),
 (x^2 + x + 1, x^2 + x + 1),
 (x^2 + 1, x^2 + x),
 (1, x^2 + x + 1)]

In [470]:
testCurve([(k, F(1) * k + fi^2 * k^2 + fi * k^4) for k in F])

True

In [473]:
(x^3)^2 == x^6

True

Now we test the $(2,3,4)$ bundle.

In [478]:
curves234_map = [
    lambda k: F(0),
    lambda k: x^6 * k + x^3 * k^2 + x^5 * k^4,
    lambda k: x^2 * k + x^5 * k^2 + x^6 * k^4,
    lambda k: x^4 * k + x^3 * k^2 + x^5 * k^4,
    lambda k: x^3 * k,
    lambda k: x^5 * k + x^5 * k^2 + x^6 * k^4,
    lambda k: x * k + x^2 * k^2 + x * k^4,
    lambda k: k + x^2 * k^2 + x * k^4
]
curves234 = [[(F(0),k) for k in F]]
curves234 = curves234 + [[(k,curve(k)) for k in F] for curve in curves234_map]

In [479]:
for curve in curves234:
    checkAdditivity(curve)
    checkIsotropy(curve)

In [480]:
testCurveSet(curves234)

Testing curve #0...
ABELIAN.
Testing curve #1...
ABELIAN.
Testing curve #2...
Testing curve #3...
Testing curve #4...
Testing curve #5...
ABELIAN.
Testing curve #6...
Testing curve #7...
ABELIAN.
Testing curve #8...


In [483]:
[(k, curves234_map[-2](k)) for k in F]

[(0, 0),
 (x, x^2 + 1),
 (x^2, x),
 (x + 1, 1),
 (x^2 + x, x^2 + x + 1),
 (x^2 + x + 1, x + 1),
 (x^2 + 1, x^2 + x),
 (1, x^2)]

In [485]:
powers([(k, curves234_map[-2](k)) for k in F])

[(0, 0), (1, 6), (2, 1), (3, 7), (4, 5), (5, 3), (6, 4), (7, 2)]

In [406]:
WignerMatrix(StateFromCurve(curves234[5]))

[1/8   0   0   0   0   0   0   0]
[  0   0   0   0 1/8   0   0   0]
[  0   0   0   0   0 1/8   0   0]
[  0   0   0   0   0   0 1/8   0]
[  0   0   0   0   0   0   0 1/8]
[  0 1/8   0   0   0   0   0   0]
[  0   0 1/8   0   0   0   0   0]
[  0   0   0 1/8   0   0   0   0]

Now we test $(0,9,0)$ curves.

In [486]:
# regular curves
curves090 = [
    [(x^2 * b + x^3 * b^2 + x^5 * b^4, b) for b in F],
    [(a, x^2 * a + x^3 * a^2 + x^5 * a^4) for a in F],
    [(b + x^6 * b^2 + x^3 * b^4, b) for b in F],
    [(x^3 * b^2 + x^5 * b^4, b) for b in F],
    [(x^6 * b + x^3 * b^2 + x^5 * b^4, b) for b in F],
    [(a, x^6 * a^2 + x^3 * a^4) for a in F],
    [(a, a + x^3 * a^2 + x^5 * a^4) for a in F]
]

degen_1 = []
for a in F:
    for b in F:
        if b^2 + x^5 * b == x^2 * a^2 + x^6 * a:
            if (x^4 * b).trace() == 0:
                if (x^5 * a).trace() == 0:
                    degen_1.append((a,b))

degen_2 = []
for a in F:
    for b in F:
        if b^2 + x^2 * b == x^6 * a^2 + x^5 * a:
            if (x^6 * b).trace() == 0:
                if (x^2 * a).trace() == 0:
                    degen_2.append((a,b))

curves090 = curves090 + [degen_1] + [degen_2]

In [487]:
for curve in curves090:
    checkAdditivity(curve)
    checkIsotropy(curve)

In [497]:
for curve1 in curves090:
    for curve2 in curves090:
        if curve1 != curve2:
            if len(list(set(curve1) & set(curve2))) != 1:
                raise Exception('Found non-trivial intersection!')

In [498]:
testCurveSet(curves090)

Testing curve #0...
Testing curve #1...
Testing curve #2...
ABELIAN.
Testing curve #3...
ABELIAN.
Testing curve #4...
Testing curve #5...
ABELIAN.
Testing curve #6...
ABELIAN.
Testing curve #7...
ABELIAN.
Testing curve #8...
ABELIAN.


In [509]:
powers(curves090[8])

[(0, 0), (0, 2), (2, 3), (2, 5), (6, 0), (6, 2), (7, 3), (7, 5)]

In [411]:
WignerMatrix(StateFromCurve(curves090[-1]))

[1/8   0 1/8   0   0   0   0   0]
[  0   0   0   0   0   0   0   0]
[  0   0   0 1/8   0 1/8   0   0]
[  0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0]
[1/8   0 1/8   0   0   0   0   0]
[  0   0   0 1/8   0 1/8   0   0]

In [412]:
graph(curves090[2])

[1 0 0 0 1 0 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 1 0]
[0 0 0 0 0 1 0 1]
[0 0 0 0 0 0 0 0]
[0 1 1 0 0 0 0 0]

In [416]:
x^4 + x^6 * (x^4)^2 + x^3 * (x^4)^4

0

In [413]:
graph(curves090[3])

[1 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 1]
[0 1 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 1 1 0 0 0]
[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]

In [414]:
graph(curves090[6])

[1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1]
[1 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 1 0]